In [63]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.preprocessing import LabelEncoder
import sklearn
import os
from matplotlib.pyplot import hist
import scipy.stats as stats
import math
import statsmodels.api as sm


In [64]:
# read in the dataframe
df = pd.read_csv('df_mix.csv')

In [65]:
df1 = pd.read_csv('../../data/GVC_data/transportIV_file.csv')
df1 = df1.loc[:, ['country', 't', 'trans_outp_p']]

df = pd.merge(df, df1, on=['country', 't'])

In [66]:
print(df.dtypes)

country                        object
t                               int64
onset2COWCS                   float64
milexp_pergdpSIPRI            float64
decade                        float64
democracy                     float64
logmountain                   float64
ethnic_fractionalization      float64
religion_fractionalization    float64
language_fractionalization    float64
leg_british                   float64
opec                          float64
logpop_M_diff                 float64
logpopdens_diff               float64
logoutreg_diff                float64
ecgrowth_demeaned             float64
treat_agri                    float64
treat_mine                    float64
treat_fuel                    float64
treat_metal                   float64
iv_transport                  float64
iv_agri                       float64
iv_mine                       float64
iv_fuel                       float64
iv_metal                      float64
trans_outp_p                  float64
dtype: objec

In [67]:
num_countries = df['country'].nunique()
print(f"There are {num_countries} unique countries in the dataset.")

There are 181 unique countries in the dataset.


Without Country Fixed effect

In [68]:
import pandas as pd
import numpy as np
import doubleml as dml
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import clone

# Define the machine learning model for the treatment effect (ml_m) and outcome (ml_l)
learner = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=1111)
ml_m = clone(learner)  # Machine learning model for the treatment effect
ml_l = clone(learner)  # Machine learning model for the outcome

In [69]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import clone
import doubleml as dml

def run_plr(df, outcome_l, treatment_l, block_l, fe, stationary_c):
    # Select the relevant columns and drop NA
    df_subset = df[outcome_l + treatment_l + block_l].dropna()

    # Prepare the data for DoubleML
    obj_dml_data = dml.DoubleMLData(df_subset, y_col=outcome_l[0], d_cols=treatment_l[0], x_cols=block_l)

    # Define the machine learning models
    learner = RandomForestRegressor(n_estimators=500, max_features='sqrt', random_state=1111)
    ml_m = clone(learner)
    ml_l = clone(learner)

    # Set up the PLR model
    dml_plr = dml.DoubleMLPLR(obj_dml_data, ml_l=ml_l, ml_m=ml_m, n_folds=10)

    # Fit the PLR model and get the summary
    plr_results = dml_plr.fit()
    summary = plr_results.summary  # Access summary as an attribute, not a method

    return summary

In [70]:
def run_all(df, outcome_l, treatment_l, block_fe_l, block_sta_l, block_other_l):
    # Adjusted the columns to store summaries in a list since they're DataFrames now
    summaries = []

    for fe in [True, False]:
        block_l = block_other_l.copy()
        if fe:
            # Exclude 'country' from fixed effects
            block_l += [b for b in block_fe_l if b != 'country']
        for sta in [True, False]:
            if sta:
                block_l += block_sta_l
            summary = run_plr(df, outcome_l, treatment_l, block_l, fe, sta)
            summaries.append(summary)  # Append the summary DataFrame to the list

    return summaries  # Return the list of summary DataFrames

In [71]:
plr_summaries = run_all(df, 
                        outcome_l=['milexp_pergdpSIPRI'], 
                        treatment_l=['treat_fuel'],
                        block_fe_l=['country', 't'],  # 'country' will be ignored in the fixed effects
                        block_sta_l=['logmountain', 'ethnic_fractionalization', 'religion_fractionalization', 'language_fractionalization'], 
                        block_other_l=['democracy', 'logpopdens_diff', 'ecgrowth_demeaned'])

In [72]:
print(plr_summaries)

[                 coef    std err         t    P>|t|      2.5 %     97.5 %
treat_fuel  15.671713  16.543013  0.947331  0.34347 -16.751997  48.095423,                  coef    std err         t     P>|t|      2.5 %     97.5 %
treat_fuel  15.905606  15.819982  1.005412  0.314698 -15.100989  46.912201,                 coef    std err         t     P>|t|      2.5 %     97.5 %
treat_fuel  7.892674  13.104323  0.602295  0.546977 -17.791327  33.576676,                  coef    std err         t     P>|t|      2.5 %     97.5 %
treat_fuel  15.260358  16.548607  0.922154  0.356448 -17.174315  47.695032]


In [73]:
import pandas as pd

# Assuming plr_summaries is a list of DataFrames as shown in your print output
# We'll add a multi-index or additional columns to indicate whether fixed effects or stationary controls were used

# Define the configurations corresponding to your summaries for clarity
configurations = [
    {'fixed_effects': True, 'stationary_controls': True},
    {'fixed_effects': True, 'stationary_controls': False},
    {'fixed_effects': False, 'stationary_controls': True},
    {'fixed_effects': False, 'stationary_controls': False},
]

# Add a multi-index or new columns to each DataFrame to indicate the configuration
for i, summary in enumerate(plr_summaries):
    summary['fixed_effects'] = configurations[i]['fixed_effects']
    summary['stationary_controls'] = configurations[i]['stationary_controls']

# Concatenate all the DataFrames into one
combined_summary = pd.concat(plr_summaries, axis=0)

# Reset index if you want a clean index
combined_summary.reset_index(drop=True, inplace=True)

# Now you can print or display the combined summary
print(combined_summary)


        coef    std err         t     P>|t|      2.5 %     97.5 %  \
0  15.671713  16.543013  0.947331  0.343470 -16.751997  48.095423   
1  15.905606  15.819982  1.005412  0.314698 -15.100989  46.912201   
2   7.892674  13.104323  0.602295  0.546977 -17.791327  33.576676   
3  15.260358  16.548607  0.922154  0.356448 -17.174315  47.695032   

   fixed_effects  stationary_controls  
0           True                 True  
1           True                False  
2          False                 True  
3          False                False  


With country fix effect

In [75]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import clone
import doubleml as dml

def run_plr(df, outcome_l, treatment_l, block_l, fe, stationary_c):
    # If fixed effects include 'country', apply one-hot encoding
    if fe and 'country' in block_l:
        df = pd.get_dummies(df, columns=['country'])
        block_l = [col for col in df.columns if col not in outcome_l + treatment_l]  # Update block_l to include new dummy variables

    # Select the relevant columns and drop NA
    df_subset = df[outcome_l + treatment_l + block_l].dropna()

    # Prepare the data for DoubleML
    obj_dml_data = dml.DoubleMLData(df_subset, y_col=outcome_l[0], d_cols=treatment_l[0], x_cols=block_l)

    # Define the machine learning models
    learner = RandomForestRegressor(n_estimators=500, max_features='sqrt', random_state=1111)
    ml_m = clone(learner)
    ml_l = clone(learner)

    # Set up the PLR model
    dml_plr = dml.DoubleMLPLR(obj_dml_data, ml_l=ml_l, ml_m=ml_m, n_folds=10)

    # Fit the PLR model and get the summary
    plr_results = dml_plr.fit()
    summary = plr_results.summary

    return summary

def run_all(df, outcome_l, treatment_l, block_fe_l, block_sta_l, block_other_l):
    # Adjusted the columns to store summaries in a list since they're DataFrames now
    summaries = []

    for fe in [True, False]:
        block_l = block_other_l.copy()
        if fe:
            block_l += block_fe_l  # Keep 'country' in block_fe_l to include country fixed effects
        for sta in [True, False]:
            if sta:
                block_l += block_sta_l
            summary = run_plr(df, outcome_l, treatment_l, block_l, fe, sta)
            summaries.append(summary)  # Append the summary DataFrame to the list

    return summaries

# Example usage
plr_summaries = run_all(df, 
                        outcome_l=['milexp_pergdpSIPRI'], 
                        treatment_l=['treat_fuel'],
                        block_fe_l=['country', 't'],  # Include 'country' for fixed effects
                        block_sta_l=['logmountain', 'ethnic_fractionalization', 'religion_fractionalization', 'language_fractionalization'], 
                        block_other_l=['democracy', 'logpopdens_diff', 'ecgrowth_demeaned'])


In [76]:
print(plr_summaries)

[                 coef    std err         t     P>|t|      2.5 %     97.5 %
treat_fuel  30.365372  30.827964  0.984994  0.324627 -30.056328  90.787071,                 coef    std err         t     P>|t|      2.5 %     97.5 %
treat_fuel  27.96129  29.873203  0.935999  0.349274 -30.589113  86.511693,                 coef    std err         t     P>|t|      2.5 %     97.5 %
treat_fuel  8.762658  14.191992  0.617437  0.536947 -19.053135  36.578452,                coef    std err         t     P>|t|      2.5 %     97.5 %
treat_fuel  6.20619  10.518196  0.590043  0.555162 -14.409095  26.821474]


In [80]:
import pandas as pd

# Define the configurations corresponding to your summaries for clarity
configurations = [
    {'fixed_effects': True, 'stationary_controls': True},
    {'fixed_effects': True, 'stationary_controls': False},
    {'fixed_effects': False, 'stationary_controls': True},
    {'fixed_effects': False, 'stationary_controls': False},
]

# Add a multi-index or new columns to each DataFrame to indicate the configuration
for i, summary in enumerate(plr_summaries):
    summary['fixed_effects'] = configurations[i]['fixed_effects']
    summary['stationary_controls'] = configurations[i]['stationary_controls']

# Concatenate all the DataFrames into one
combined_summary = pd.concat(plr_summaries, axis=0)

# # Reset index to obtain a clean index
combined_summary.reset_index(drop=True, inplace=True)
rounded_combsum = round(combined_summary, 4)

print(rounded_combsum)


      coef  std err       t   P>|t|    2.5 %   97.5 %  fixed_effects  \
0  30.3654  30.8280  0.9850  0.3246 -30.0563  90.7871           True   
1  27.9613  29.8732  0.9360  0.3493 -30.5891  86.5117           True   
2   8.7627  14.1920  0.6174  0.5369 -19.0531  36.5785          False   
3   6.2062  10.5182  0.5900  0.5552 -14.4091  26.8215          False   

   stationary_controls  
0                 True  
1                False  
2                 True  
3                False  


In [81]:
dfs = [rounded_combsum]
stacked_df = pd.concat(dfs)
final_res = stacked_df.reset_index(drop=True)
final_res.insert(0, 'gvc_type', 'mix')
final_res
final_res.to_csv('mix_res.csv', index=False)